## Imports

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from models.mog_model import *
from utils.distributions import *
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice
from lattice.fields import SU2, SU3
from utils.dynamics import *
from utils.gauge_dynamics import GaugeDynamics
from utils.func_utils import accept, jacobian, autocovariance, get_log_likelihood, binarize, normal_kl, acl_spectrum, ESS
from utils.network import network, Linear, Sequential, Zip, Parallel, ScaleTanh
from utils.sampler import propose
from utils.notebook_utils import get_hmc_samples
from utils.data_utils import calc_avg_vals_errors, block_resampling, jackknife_err
from matplotlib import gridspec
from utils.plot_helper import errorbar_plot
plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')

plt.rc('text', usetex=False)
plt.rcParams['errorbar.capsize'] = 0
%matplotlib notebook
%autoreload 2

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
def restart_tf(sess):
    tf.reset_default_graph()
    sess.close()
    return tf.InteractiveSession()

## Create GaugeLattice

In [ ]:
sess = restart_tf(sess)

In [ ]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = lattice.get_links_samples(batch_size, link_type='SU3')

In [ ]:
dynamics = GaugeDynamics(lattice, trajectory_length=2, eps=0.1,
                         hmc=False, net_factory=network, eps_trainable=True,
                         use_temperature=True)

In [ ]:
link = tf.placeholder(dtype=tf.complex64, shape=lattice.link_shape, name='link')
x_shape = tuple([batch_size] + list(lattice.links.shape))
x = tf.placeholder(tf.complex64, shape=x_shape, name='x')
z = tf.random_normal(tf.shape(x), name='z')

In [ ]:
energy = dynamics.energy(x, batch_size)

In [ ]:
energy_arr = sess.run(energy, feed_dict={x: links_samples, dynamics.temperature: 1.})

In [ ]:
energy_arr

In [ ]:
sess.run(dynamics.kinetic(z))

In [ ]:
H = dynamics.hamiltonian(x, z)
print(sess.run(H), feed_dict={x: samples, dynamics.temperature: 1.})

In [ ]:
step1 = dynamics._forward_step(x, z, 1)

In [ ]:
%debug

In [ ]:
Lx, _, px, output = propose(x, dynamics, do_mh_step=True)

In [ ]:
Lz, _, pz, _ = propose(z, dynamics, do_mh_step=True)

loss = tf.Variable(0., trainable=False, name='loss')
v1 = ((tf.reduce_sum(tf.square(x - Lx), axis=1) * px) + 1e-4)
v2 = ((tf.reduce_sum(tf.square(z - Lz), axis=1) * pz) + 1e-4)
scale = 0.1
loss = loss + scale * (tf.reduce_mean(1.0 / v1) + tf.reduce_mean(1.0 / v2))
loss = loss + ((-tf.reduce_mean(v1, name='v1')
                - tf.reduce_mean(v2, name='v2')) / scale)

In [ ]:
with tf.Session() as sess:
    print(sess.run(lattice.total_action(lattice.links)))

In [ ]:
energy = dynamics.energy(x)
with tf.Session() as sess:
    print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature:1.}))

### Visualize TF graph

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=4096):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=4096):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))


In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
%debug

## LatticeDynamics testing

### Init tf.Session()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.close()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
sess.run(tf.global_variables_initializer())

### Create LatticeDynamics object

In [ ]:
_x_shape = tuple([None] + list(lattice.links.shape) + list(lattice.link_shape))
_x = tf.placeholder(dtype=tf.complex64, shape=_x_shape)

_links_samples = []
for _ in range(10):
    _links_sample = np.zeros(_x_shape[1:], dtype=np.complex64)
    for link in lattice.iter_links():
        _links_sample[link] = lattice.link_type.get_random_element()
    _links_samples.append(_links_sample)

In [ ]:
lattice.num_links

In [ ]:
_mask = sess.run(dynamics.mask)

In [ ]:
_mask.reshape(tuple([-1] + list(lattice.links.shape)))

In [ ]:
m, _m = dynamics._get_mask(0)

In [ ]:
np.array(links_samples).astype(np.complex64)

In [ ]:
links_samples = np.array(np.array(links_samples), dtype=np.complex64)

In [ ]:
kinetic = dynamics.kinetic(z)
print(sess.run(kinetic), feed_dict={x: np.array(links_samples), dynamics.temperature: 1.})

In [ ]:
hamiltonian = dynamics.hamiltonian(x, z)
sess.run(hamiltonian, feed_dict{x:})

In [ ]:
temp = np.array([1.0], dtype=np.float32)[0]
energy = dynamics.energy(x)
print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature: temp}))

In [ ]:
print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature: temp}))

In [ ]:
print(sess.run(dynamics.lattice.total_action()))
#dynamics.lattice.total_action()

In [ ]:
forward_step = dynamics._forward_step(x, z, 1, aux=None)

In [ ]:
%debug

In [ ]:
step1 = sess.run(forward_step, feed_dict={x: samples, dynamics.temperature: 2.})

In [ ]:
x1, v1, log_jac_contrib1 = step1

In [ ]:
energy = dynamics.energy(x)
print(sess.run(energy, feed_dict={x: samples, dynamics.temperature:2.}))
print(sess.run(energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
hamiltonian = dynamics.hamiltonian(x, z)
print(sess.run(hamiltonian, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(hamiltonian, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
grad_energy = dynamics.grad_energy(x)
print(sess.run(grad_energy, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(grad_energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
X, V, p_accept = dynamics.forward(x)

In [ ]:
_X = sess.run(X, feed_dict={x: samples, dynamics.temperature:2.})
print(_X.shape)

In [ ]:
_V = sess.run(V, feed_dict={x: samples, dynamics.temperature:2.})
print(_V.shape)

In [ ]:
_p_accept = sess.run(p_accept, feed_dict={x: samples, dynamics.temperature:2.})
print(_p_accept.shape)
print(_p_accept)